In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque
from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate
import os
from sklearn.model_selection import train_test_split
import pickle
from surprise import accuracy
from surprise import accuracy, Dataset, Reader, SVD, KNNBaseline
from surprise.model_selection import PredefinedKFold
import pprint
import tempfile
from typing import Dict, Text
import tensorflow as tf
import tensorflow_recommenders as tfrs
from collections import defaultdict
import joblib
from scipy import stats

In [2]:
PATH = '/Users/alecclarkfeather/capstone_project/'

In [3]:
def five_by_two_paired_t_test(df, col1, col2):
    variances = []
    diffs = []
    for i in range(2, 12, 2):
        group1 = df[col1].iloc[:i].values
        group2 = df[col2].iloc[:i].values
        result1a = group1[i-2]
        result1b = group1[i-1]
        result2a = group2[i-2]
        result2b = group2[i-1]
        performance1 = result1a - result2a
        performance2 = result1b - result2b
        mean_performance = (performance1 + performance2) / 2
        variance_performance = (performance1 - mean_performance)**2 + (performance2 - mean_performance)**2
        variances.append(variance_performance)
        diffs.append(performance1)
        
    t_statistic = diffs[0] / np.sqrt((1/len(variances))*np.sum(variances))
    p_value = stats.t.sf(abs(t_statistic), df=len(variances))
    
    return t_statistic, p_value
        

In [4]:
rmse_nn_df = pd.read_csv(PATH + 'rmse_nn_df.csv')
rmse_nn_df

,neural_network_rmse
0,1.031356
1,1.024157
2,1.033121
3,1.028123
4,1.031856
5,1.021747
6,1.031770
7,1.024254
8,1.033011
9,1.026739


In [5]:
rmse_results_df = pd.read_csv(PATH + 'rmse_results_df.csv')
rmse_results_df

,baseline,cf,svd
0,1.478618,1.066947,1.006683
1,1.477799,1.067403,1.006872
2,1.478326,1.067594,1.007039
3,1.478533,1.067073,1.007092
4,1.478335,1.066790,1.007514
5,1.477217,1.066989,1.006884
6,1.477425,1.067445,1.006484
7,1.478469,1.067561,1.007107
8,1.477504,1.067057,1.007069
9,1.478085,1.067257,1.006919


In [6]:
rmse_results_df_final = pd.concat([rmse_results_df, rmse_nn_df], axis=1)
rmse_results_df_final

,baseline,cf,svd,neural_network_rmse
0,1.478618,1.066947,1.006683,1.031356
1,1.477799,1.067403,1.006872,1.024157
2,1.478326,1.067594,1.007039,1.033121
3,1.478533,1.067073,1.007092,1.028123
4,1.478335,1.066790,1.007514,1.031856
5,1.477217,1.066989,1.006884,1.021747
6,1.477425,1.067445,1.006484,1.031770
7,1.478469,1.067561,1.007107,1.024254
8,1.477504,1.067057,1.007069,1.033011
9,1.478085,1.067257,1.006919,1.026739


In [7]:
t_test_results_rmse_df = pd.read_csv(PATH + 't_test_results_rmse_df.csv')
t_test_results_rmse_df

,t_statistic,p_value_unadjusted
0,588.374640,1.345832e-13
1,1058.543343,7.140465e-15
2,157.250991,9.865492e-11


In [8]:
index=['baseline_vs_cf','baseline_vs_svd','cf_vs_svd']
t_test_results_rmse_df.index = index
t_test_results_rmse_df

,t_statistic,p_value_unadjusted
baseline_vs_cf,588.374640,1.345832e-13
baseline_vs_svd,1058.543343,7.140465e-15
cf_vs_svd,157.250991,9.865492e-11


In [9]:
nn_rmse_t_stats = []
nn_rmse_p_vals = []

t_b_vs_nn, p_b_vs_nn = five_by_two_paired_t_test(rmse_results_df_final, 'baseline', 'neural_network_rmse')
t_cf_vs_nn, p_cf_vs_nn = five_by_two_paired_t_test(rmse_results_df_final, 'cf', 'neural_network_rmse')
t_svd_vs_nn, p_svd_vs_nn = five_by_two_paired_t_test(rmse_results_df_final, 'svd', 'neural_network_rmse')

nn_rmse_t_stats.append(t_b_vs_nn)
nn_rmse_t_stats.append(t_cf_vs_nn)
nn_rmse_t_stats.append(t_svd_vs_nn)

nn_rmse_p_vals.append(p_b_vs_nn)
nn_rmse_p_vals.append(p_cf_vs_nn)
nn_rmse_p_vals.append(p_svd_vs_nn)

In [10]:
nn_rmse_t_stats

[86.25609332643153, 6.6666985618208585, -4.716380452651623]

In [11]:
nn_rmse_p_vals

[1.984722225383396e-09, 0.0005732410979249241, 0.0026297893205232134]

In [12]:
t_test_results_dict_nn_rmse = dict(t_statistic=nn_rmse_t_stats, p_value_unadjusted=nn_rmse_p_vals)
t_test_results_dict_nn_rmse

{'t_statistic': [86.25609332643153, 6.6666985618208585, -4.716380452651623],
 'p_value_unadjusted': [1.984722225383396e-09,
  0.0005732410979249241,
  0.0026297893205232134]}

In [13]:
t_test_results_rmse_nn_df = pd.DataFrame(t_test_results_dict_nn_rmse, index=['baseline_vs_neural_network',
                                                                 'cf_vs_neural_network',
                                                                 'svd_vs_neural_network'])
t_test_results_rmse_nn_df

,t_statistic,p_value_unadjusted
baseline_vs_neural_network,86.256093,1.984722e-09
cf_vs_neural_network,6.666699,5.732411e-04
svd_vs_neural_network,-4.716380,2.629789e-03


In [14]:
t_test_results_rmse_final = pd.concat([t_test_results_rmse_df, t_test_results_rmse_nn_df], axis=0)
t_test_results_rmse_final

,t_statistic,p_value_unadjusted
baseline_vs_cf,588.374640,1.345832e-13
baseline_vs_svd,1058.543343,7.140465e-15
cf_vs_svd,157.250991,9.865492e-11
baseline_vs_neural_network,86.256093,1.984722e-09
cf_vs_neural_network,6.666699,5.732411e-04
svd_vs_neural_network,-4.716380,2.629789e-03


In [15]:
original_alpha = 0.05
bonferroni_adjusted_alpha = original_alpha / 6
bonferroni_adjusted_alpha

0.008333333333333333

In [16]:
t_test_results_rmse_final['significant'] = \
t_test_results_rmse_final['p_value_unadjusted'].apply(lambda x: 'Yes' if x <= bonferroni_adjusted_alpha
                                                   else 'No')

t_test_results_rmse_final

,t_statistic,p_value_unadjusted,significant
baseline_vs_cf,588.374640,1.345832e-13,Yes
baseline_vs_svd,1058.543343,7.140465e-15,Yes
cf_vs_svd,157.250991,9.865492e-11,Yes
baseline_vs_neural_network,86.256093,1.984722e-09,Yes
cf_vs_neural_network,6.666699,5.732411e-04,Yes
svd_vs_neural_network,-4.716380,2.629789e-03,Yes


In [17]:
rmse_means = {}
rmse_stds = {}

for col in rmse_results_df_final.columns:
    rmse_means[col] = np.mean(rmse_results_df_final[col].values)
    rmse_stds[col] = np.std(rmse_results_df_final[col].values, ddof=1)


In [18]:
rmse_means

{'baseline': 1.4780310223914437,
 'cf': 1.0672115886736289,
 'svd': 1.0069664447184459,
 'neural_network_rmse': 1.028613376769359}

In [19]:
rmse_stds

{'baseline': 0.00050898120189072,
 'cf': 0.00027915446090659543,
 'svd': 0.0002753880770122459,
 'neural_network_rmse': 0.004182222257101177}

In [20]:
rmse_means_df = pd.DataFrame(rmse_means, index=['mean'])
rmse_stds_df = pd.DataFrame(rmse_stds, index=['standard deviation'])

rmse_summary_statistics_final = pd.concat([rmse_means_df, rmse_stds_df], axis=0)

rmse_summary_statistics_final

,baseline,cf,svd,neural_network_rmse
mean,1.478031,1.067212,1.006966,1.028613
standard deviation,0.000509,0.000279,0.000275,0.004182


In [21]:
rmse_summary_statistics_final.to_csv(PATH + 'rmse_summary_statistics_final.csv')
rmse_results_df_final.to_csv(PATH + 'rmse_results_df_final.csv')
t_test_results_rmse_final.to_csv(PATH + 't_test_results_rmse_final.csv')

In [22]:
sample_t, sample_p = five_by_two_paired_t_test(rmse_results_df_final, 'baseline', 'cf')
print(sample_t)
print(sample_p)

588.3746395042142
1.345831530364664e-13
